## Asignatura Procesamiento de Lenguaje Natural
### Por Elizabeth León Guzmán
------------------------------------------------------------------------------

# Minería de Texto

En muchas ocasiones es necesario construir modelos que clasifiquen automáticamente un texto dado. Para esto, en algunas oportunidades disponemos de texto anotado, es decir, documentos que fueron previamente clasificados dentro de dos o más categorías. Este tipo de información es de gran utilidad al permitir desarrollar modelos predictivos en una gran variedad de aplicaciones, por ejemplo: 

* Clasificación de spam en correos.
* Detección de depresión en redes sociales.
* Análisis de sentimientos.
* Clasificación del lenguaje de un texto.
* Identificación de noticias falsas.
* Atribución de autoría.

![classtexto](https://miro.medium.com/max/1400/0*6nPN6naiH7lApcvg.png)

La tarea de **clasificación de texto** es una aplicación típica de las técnicas de machine learning de aprendizaje supervisado.

---
## 1. Preprocesamiento

En este caso, utilizaremos NLTK para el preprocesamiento de los textos. Se define una función para eliminar caracteres especiales, stopwords, mayúsculas.

In [1]:
# Importamos las librerías básicas:
#   numpy: librería numérica de Python
#   pandas: análisis de datos
#   re: expresiones regulares
#   nltk: procesamiento de LN
#   matplotlib: visualizaciones 

import numpy as np 
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt
from collections import defaultdict
plt.style.use("ggplot")
pd.options.display.max_colwidth = 200
%matplotlib inline

In [2]:
# Descargamos recursos en inglés populares: conjuntos de datos, diccionarios, etc (wordnet, snowball, etc. )
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/juan.gama/nltk_data...
[nltk_data]    |   Package movie_

True

In [2]:
# Definimos un tokenizer que separa texto de signos de puntuación y números.
wpt = nltk.WordPunctTokenizer()
# Descargamos las stopwords para inglés
stop_words = nltk.corpus.stopwords.words('english')

In [4]:
# Definimos la función de preprocesamiento
def normalize_document(doc):
    # Se eliminan caracteres especiales
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.IGNORECASE|re.ASCII)
          # Más info compilation flags: https://docs.python.org/3/howto/regex.html#compilation-flags
    # Se convierten los textos a minúsculas
    doc = doc.lower()
    # Se elimin espacios al inicio y final del texto
    doc = doc.strip()  
    # Tokenizado de documento
    tokens = wpt.tokenize(doc)
    # Eliminación de stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Retornamos una versión filtrada del texto
    doc = ' '.join(filtered_tokens)
    return doc
# Vectorización de la función
normalize_corpus = np.vectorize(normalize_document)
# Más info: https://docs.scipy.org/doc/numpy/reference/generated/numpy.vectorize.html
# The vectorized function evaluates "normalize_document" over successive tuples 
# of the input arrays like the python map function

Ahora, se mostrará un ejemplo con un dataset sencillo:

In [3]:
# Se define un corpus de ejemplo
corpus = ['The sky is blue and beautiful.',
'Love this blue and beautiful sky!',
'The quick brown fox jumps over the lazy dog.',
"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
'I love green eggs, ham, sausages and bacon!',
'The brown fox is quick and the blue dog is lazy!',
'The sky is very blue and the sky is very beautiful today',
'The dog is lazy but the brown fox is quick!'
]
# Se asigna una etiqueta a cada texto
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']
# Creamos un dataframe de pandas con el corpus y las etiquetas
corpus_df = pd.DataFrame({"Document": corpus, "Category": labels})
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


In [5]:
norm_corpus=normalize_corpus(corpus)
norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog',
       'kings breakfast sausages ham bacon eggs toast beans',
       'love green eggs ham sausages bacon',
       'brown fox quick blue dog lazy', 'sky blue sky beautiful today',
       'dog lazy brown fox quick'], dtype='<U51')

---
## 2. Representaciones de texto

En este notebook nos enfocaremos en algunas técnicas para la **representación de textos** que son comunmente generadas después de tareas de procesamiento de lenguaje natural. En particular, estudiaremos las siguientes representaciones:

* Bolsa de palabras (Bag of Words - BoW)
* Métodos de pesado como tf-idf
* N-grams
* Word2Vec
* FastText

### 2.1 Bolsa de Palabras - Bag of Words (BoW)

Se trata de uno de los métodos de representación más simples y consiste en encontrar una distribución de todos los términos $T=\{t_1,t_2,\dots,t_l\}$ que aparecen en un conjunto de documentos $D=\{d_1,d_2,\dots,d_m\}$ por medio de técnicas de conteo, así: $P(T=t_i,D=d_j)$

![bow](https://qph.fs.quoracdn.net/main-qimg-4934f0958e121d33717f848230ef664a)


El enfoque típico para estimar una bolsa de palabras está basado en conteos, más específicamente, se encuentra una matríz:

 $\mathbf{Tf}(t_i,d_j)$ 
 
 que contiene el número de veces que se encuentra un término $t_i$ en un documento $d_j$. Al normalizar esta matriz para que la suma de las ocurrencias de todos los términos en un documento $d_j$ sea uno, se obtiene una estimación de la distribución $P(T=t_i|D=d_j)$. 

Más info: https://en.wikipedia.org/wiki/Bag-of-words_model

Ahora, veamos cómo construir una bolsa de palabras. Primero, lo haremos paso a paso (para propósitos de demostración únicamente). Segundo, usaremos librerías para hacerlo de forma óptima (así lo usaremos normalmente):

#### 2.1.1 Construcción BoW: paso a paso

**Para propósitositos didácticos únicamente.** 
En esta sección usaremos Python básico para construir un sistema NLP rudimentario. Crearemos un vocabulario a partir de todas las palabras de nuestro corpus y luego usaremos la técnica de Bolsa de palabras (BoW) para extraer características de cada documento.

1. **Construimos el vocabulario**

In [10]:
print(norm_corpus)

['sky blue beautiful' 'love blue beautiful sky'
 'quick brown fox jumps lazy dog'
 'kings breakfast sausages ham bacon eggs toast beans'
 'love green eggs ham sausages bacon' 'brown fox quick blue dog lazy'
 'sky blue sky beautiful today' 'dog lazy brown fox quick']


In [26]:
vocab = set(' '.join(norm_corpus).split(' '))
vocab = { word: idx for idx, word in enumerate(vocab)}
print(vocab)

{'breakfast': 0, 'beautiful': 1, 'lazy': 2, 'love': 3, 'dog': 4, 'sausages': 5, 'fox': 6, 'green': 7, 'beans': 8, 'quick': 9, 'today': 10, 'eggs': 11, 'jumps': 12, 'toast': 13, 'brown': 14, 'bacon': 15, 'blue': 16, 'kings': 17, 'ham': 18, 'sky': 19}


2. **Extración de características**

In [52]:
# Creamos una lista vacía para cada documento con una posición para cada palabra en el vocabulario
shape = (len(norm_corpus), len(vocab)) 
tf = np.zeros(shape, np.int32)
print(tf)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [44]:
tf[0][6]

0

In [53]:
# Mapeamos las frecuencias de cada palabra en tf:
i = 0
for doc in norm_corpus:
  for word in doc.split():
    tf[i][vocab[word]] += 1
  i += 1

print(tf)

[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 1 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0]
 [0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0]
 [0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 2]
 [0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0]]


Por ejemplo:

In [57]:
print(norm_corpus[0], tf[0])

sky blue beautiful [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]


In [58]:
print(norm_corpus[6])
print(tf[6])

sky blue sky beautiful today
[0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 2]


Hay que tener en cuenta que si nuestros documentos fueran consideramente más largos y estuviéramos analizando un conjunto mayor de documentos, solamente algunos elementos en la matriz tendrían valores. El vocabulario sería muchísimo más grande y las listas contendrían principalmente valores en cero. Serían **matrices dispersas** con muchas dimensiones (*sparse*).

#### 2.1.2 Construcción BoW: usando scikit-learn

In [59]:
# Importamos el extractor de características BoW de Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
# Instanciamos el extractor de características para la ocurrencia de palabras
cv = CountVectorizer()
# Extraemos las características del corpus 
cv_matrix = cv.fit_transform(norm_corpus)
# El resultado es una matriz sparse
cv_matrix

<8x20 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [67]:
# Convertimos de la representación dispersa (sparse) a la representación densa 
# para visualizarla como numpy array
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]])

Como podemos observar cada documento es representado como una fila y cada columna representa una palabra en el corpus. En este caso los términos del vocabulario están ordenados en orden alfabético. Podemos visualizarlo mejor usando un pandas dataframe.

In [68]:
# Obtenemos todas las palabras diferentes en el corpus
vocab = cv.get_feature_names()
# Mostramos el documento y las features
tf_df = pd.DataFrame(cv_matrix, columns=vocab)
tf_df

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,0,0
3,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0
4,1,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,0,0,0
5,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1
7,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,1,0,0,0,0


Por ejemplo:

In [70]:
print(norm_corpus[6], '\n')
print(tf_df.loc[6])

sky blue sky beautiful today 

bacon        0
beans        0
beautiful    1
blue         1
breakfast    0
brown        0
dog          0
eggs         0
fox          0
green        0
ham          0
jumps        0
kings        0
lazy         0
love         0
quick        0
sausages     0
sky          2
toast        0
today        1
Name: 6, dtype: int64


Se puede ver claramente que cada columna o dimensión representa una palabra en el corpus y cada fila representa un documento. Cada celda representa el número de veces que una palabra (columna) aparece en un documento (fila).


**IMPORTANTE:** Al utilizar la representación de Bolsa de Palabras, se recomienda normalizar los valores de las frecuencias de los términos. Para esto, se divide el número de apariciones de una palabra por el número total de palabras en el documento. De esta forma, la cantidad de veces que aparece una palabra en documentos grandes se puede comparar con la de documentos más pequeños.



### 2.2 TF-IDF

Una de las principales desventajas de las bolsas de palabras es que asumen que todos los términos tienen igual importancia. No obstante, existen dos casos en los que algunos términos deberían tener mayor o menor importancia:

1. Palabras comunes que aparecen en todos los documentos y no aportan mucha información para distinguir un documento de otro. 
2. Términos únicos y poco frecuentes que son sumamente relevantes para distinguir algunos documentos en específico.

En este ámbito, surge la necesidad de estrategias que permitan capturar este tipo de relaciones. Una de las soluciones más comunes es **term frequency - inverse document frequency (TF-IDF)**. Se trata de un método que fue propuesto como métrica para la evaluación de resultados en motores de búsqueda y se convirtió en un estándar dentro de los sistemas de recuperación de información.


**TF-IDF** amplía la idea de bolsas de palabras al ponderar cada término $t_i$ por un peso $w_{i}$ como se muestra a continuación:

$$
tfidf(t_i|d_j)=\mathbf{Tf}(t_i,d_j)w_i
$$

Con esto, se puede asignar un peso menor a aquellos términos comunes entre documentos y un peso mayor a aquellos términos poco frecuentes. Una de las formas más comunes para determinar estos pesos es con la **frecuencia inversa de documento** $w_{idf}(t_i)$ que se calcula así:

$$
w_{idf}(t_i)=1+\log{\frac{N}{1+df(t_i)}}
$$

Donde $N$ es el número de documentos en el corpus y $df(t_i)$ es el número de documentos en los que se encuentra el término $t_i$.

Adicionalmente, una extención de TF-IDF consiste en un cambio de escala de la matríz $\mathbf{Tf}$, con esto, se busca atenuar el impacto que tienen los términos que aparecen muchas veces en un documento. Esto se consigue utilizando *sub-linear scaling* $wf(\mathbf{Tf}(t_i,d_j))$ y consiste en transformar las ocurrencias a una escala logarítmica donde los valores grandes se ven atenuados:

$$
wf(\mathbf{Tf}(t_i,d_j)) = \left\{
	       \begin{array}{cl}
		 1+\log{\mathbf{Tf}(t_i,d_j)}      & \mathrm{si\ } \mathbf{Tf}(t_i,d_j) > 0 \\
		 0 & \mathrm{si\ } \mathbf{Tf}(t_i,d_j) \le 0 \\
	       \end{array}
	     \right.
$$

Finalmente, una versión de TF-IDF con *sub-linear scaling* se muestra a continuación:

$$
tfidf_{2}(t_i|d_j)=wf(\mathbf{Tf}(t_i,d_j))w_{idf}(t_i)
$$

Más info: [**tf-idf**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

Veamos cómo obtener una representación TF-IDF usando scikit-learn:


In [71]:
# Importamos el extractor de características tf-idf de scikit-learn 
from sklearn.feature_extraction.text import TfidfTransformer

In [76]:
# Creamos el objeto tf-idf transformer
tt = TfidfTransformer(use_idf=True, sublinear_tf=True)
# Obtenemos la matríz tf-idf utilizando la bolsa de palabras que extragimos anteriormente 
tt_matrix = tt.fit_transform(cv_matrix)
tt_matrix

<8x20 sparse matrix of type '<class 'numpy.float64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [77]:
# Mostramos la representación tf-idf
tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names()
pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0.00,0.00,0.60,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00,0.00
1,0.00,0.00,0.49,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.49,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.38,0.38,0.00,0.38,0.00,0.00,0.53,0.00,0.38,0.00,0.38,0.00,0.00,0.00,0.00
3,0.32,0.38,0.00,0.00,0.38,0.00,0.00,0.32,0.00,0.00,0.32,0.00,0.38,0.00,0.00,0.00,0.32,0.00,0.38,0.00
4,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.00,0.47,0.39,0.00,0.00,0.00,0.39,0.00,0.39,0.00,0.00,0.00
5,0.00,0.00,0.00,0.37,0.00,0.42,0.42,0.00,0.42,0.00,0.00,0.00,0.00,0.42,0.00,0.42,0.00,0.00,0.00,0.00
6,0.00,0.00,0.39,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.00,0.54
7,0.00,0.00,0.00,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.00,0.00,0.00,0.45,0.00,0.45,0.00,0.00,0.00,0.00


`TfidfTransformer` toma como entrada el vector de Bag of Words y lo transforma en su representación tf-idf. 

**IMPORTANTE**: No es necesario calcular el modelo BoW antes de obtener la representación tf-idf. Con la clase `TfidfVectorizer` podremos obtener la representación tf-idf directamente del texto como se muestra a continuación. 

In [78]:
# Importamos el extractor de características de scikit-learn 
from sklearn.feature_extraction.text import TfidfVectorizer

In [79]:
# Instanciamos la clase 
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True, sublinear_tf=True)
# Entrenamos y extraemos características 
tv_matrix= tv.fit_transform(norm_corpus)
tv_matrix

<8x20 sparse matrix of type '<class 'numpy.float64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [80]:
# Visualizamos el resultado tf-idf obtenido
tf_idf_matrix= tv_matrix.toarray()
vocab = tv.get_feature_names()
pd.DataFrame(np.round(tf_idf_matrix, 2), columns=vocab)

,bacon,beans,beautiful,blue,breakfast,brown,dog,eggs,fox,green,ham,jumps,kings,lazy,love,quick,sausages,sky,toast,today
0,0.00,0.00,0.60,0.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00,0.00
1,0.00,0.00,0.49,0.43,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.57,0.00,0.00,0.49,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.38,0.38,0.00,0.38,0.00,0.00,0.53,0.00,0.38,0.00,0.38,0.00,0.00,0.00,0.00
3,0.32,0.38,0.00,0.00,0.38,0.00,0.00,0.32,0.00,0.00,0.32,0.00,0.38,0.00,0.00,0.00,0.32,0.00,0.38,0.00
4,0.39,0.00,0.00,0.00,0.00,0.00,0.00,0.39,0.00,0.47,0.39,0.00,0.00,0.00,0.39,0.00,0.39,0.00,0.00,0.00
5,0.00,0.00,0.00,0.37,0.00,0.42,0.42,0.00,0.42,0.00,0.00,0.00,0.00,0.42,0.00,0.42,0.00,0.00,0.00,0.00
6,0.00,0.00,0.39,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.66,0.00,0.54
7,0.00,0.00,0.00,0.00,0.00,0.45,0.45,0.00,0.45,0.00,0.00,0.00,0.00,0.45,0.00,0.45,0.00,0.00,0.00,0.00


#### 2.2.1 Medidas de similitud: Norma euclidiana

Como se mencionó anteriormente, **TF-IDF** es un método que surgió en sistemas de recuperación de información, ya que, **permite representar un documento en un espacio vectorial continúo** $\mathbb{R}^{|v|}$ donde los documentos se pueden comparar de forma numérica utilizando alguna **medida de similitud**. 

<img src="https://raw.githubusercontent.com/larajuse/share/master/vector.png" alt="vector" width="500"/>

Una de las formas más simples para comparar la representación de dos documentos $\vec{x}$ y $\vec{y}$ es por medio de la **distancia entre los vectores o la [norma euclidiana](https://en.wikipedia.org/wiki/Norm_%28mathematics%29#Euclidean_norm)** $||\vec{x}-\vec{y}||$. No obstante, esta métrica no funciona tan bien con TF-IDF, esto se debe principalmente a que en esta representación importan más los términos comunes que los términos continúos (que en una representación de bolsa de palabras, no tienen ningún orden) y las magnitudes. Veamos un ejemplo de lo anterior:

In [81]:
# definimos tres oraciones:
X = ["new in america",
     "i am new here but america is a great place to live",
     "spectrometry is an indispensable new tool"]

In [121]:
set(' '.join(X).split(' '))

{'a',
 'am',
 'america',
 'an',
 'but',
 'great',
 'here',
 'i',
 'in',
 'indispensable',
 'is',
 'live',
 'new',
 'place',
 'spectrometry',
 'to',
 'tool'}

In [94]:
# Representamos los ejemplos
vect = TfidfVectorizer(norm = None)
X_tfidf = vect.fit_transform(X).toarray()
pd.DataFrame(np.round(X_tfidf, 2), columns=vect.get_feature_names())

,am,america,an,but,great,here,in,indispensable,is,live,new,place,spectrometry,to,tool
0,0.00,1.29,0.00,0.00,0.00,0.00,1.69,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.00
1,1.69,1.29,0.00,1.69,1.69,1.69,0.00,0.00,1.29,1.69,1.0,1.69,0.00,1.69,0.00
2,0.00,0.00,1.69,0.00,0.00,0.00,0.00,1.69,1.29,0.00,1.0,0.00,1.69,0.00,1.69


Ahora, miremos cual documento está más cerca al documento 0 (más cerca significa menor distancia euclidiana).

In [84]:
d1 = np.linalg.norm(X_tfidf[0] - X_tfidf[1])
d2 = np.linalg.norm(X_tfidf[0] - X_tfidf[2])
print("La distancia euclidiana entre el documento 0 y el documento 1 es: {}".format(d1))
print("La distancia euclidiana entre el documento 0 y el documento 2 es: {}".format(d2))

La distancia euclidiana entre el documento 0 y el documento 1 es: 4.959042661645336
La distancia euclidiana entre el documento 0 y el documento 2 es: 4.201188773980275


Podemos ver que el documento 2 está más cerca al documento 0, sin embargo, fácilmente podemos ver que el documento 1 y el documento 0 son semánticamente más parecidos. Esto se debe a que la distancia euclidiana no tiene en cuenta la distribución de los términos, sólo tiene en cuenta los puntos más cercanos en este espacio.

#### 2.2.2 Medidas de similitud: coseno

Una alternativa es la **[similitud coseno](https://es.wikipedia.org/wiki/Similitud_coseno)**, la cual es más apropiada para representaciones basadas en histogramas como TF-IDF, ya que, es una medida del alineamiento de dos vectores.

$$
cosine(\vec{x},\vec{y})=\frac{\vec{x}\cdot\vec{y}}{||\vec{x}||~||\vec{y}||}
$$

Veamos el mismo ejemplo, pero con la similitud coseno:

In [98]:
from sklearn.metrics.pairwise import cosine_similarity

In [123]:
d1 = cosine_similarity(X_tfidf[0].reshape(1,-1), X_tfidf[1].reshape(1,-1)).flatten()
d2 = cosine_similarity(X_tfidf[0].reshape(1,-1),X_tfidf[2].reshape(1,-1)).flatten()
print(f"La similitud coseno entre el documento 0 y el documento 1 es {d1[0]}")
print(f"La similitud coseno entre el documento 0 y el documento 2 es {d2[0]}")

La similitud coseno entre el documento 0 y el documento 1 es 0.22901631859761276
La similitud coseno entre el documento 0 y el documento 2 es 0.11319907547001144


In [116]:
X_tfidf[0]

array([0.        , 1.28768207, 0.        , 0.        , 0.        ,
       0.        , 1.69314718, 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 0.        ])

In [0]:
X_tfidf[0].reshape(1,-1)

array([[0.        , 1.28768207, 0.        , 0.        , 0.        ,
        0.        , 1.69314718, 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

In [0]:
X_tfidf[0].reshape(1,15)

array([[0.        , 1.28768207, 0.        , 0.        , 0.        ,
        0.        , 1.69314718, 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ]])

En este caso, el documento 1 está más cerca que el documento 2 (más cerca en similitud coseno significa valores cercanos a 1, lo que equivale a un ángulo cercano a cero). En general, la similitud coseno funciona mejor en este tipo de representaciones, por ello, es la más ampliamente usada.

### 2.3 N-grams

Una alternativa a los métodos de conteo de palabras son los n-gramas, los cuales, consisten en conteos de secuencias de letras o de determinadas palabras. Es decir, se busca una distribución de secuencias $s_i$ dado un determinado documento $d_j$, así: $P(S=s_i|D=d_j)$

* Un ejemplo de secuencias a nivel de caracter puede ser $S=\{``uni",``niv",``ive", ``ver", ``ers", ``rsi", ``sid", ``ida", ``dad"\}$, que corresponden a los 3-gramas que conforman la palabra "universidad".

* Un ejemplo a nivel de secuencias a nivel de palabras puede ser: $S=\{``albert~einstein", ``einstein~era", ``era~un", ``un~cientifico"\}$, lo que corresponde a los 2-gramas que conforman la frase "albert einstein era un cientifico".

Veamos la implementación de este método:

In [124]:
# 3-grams a nivel caracter
cv = CountVectorizer(ngram_range=(3,3), analyzer="char")
# Extraemos las características del corpus 
cv_matrix = cv.fit_transform(["universidad", "universo"])

In [0]:
pd.DataFrame(data=cv_matrix.toarray(), columns=cv.get_feature_names())

,dad,ers,ida,ive,niv,rsi,rso,sid,uni,ver
0,1,1,1,1,1,1,0,1,1,1
1,0,1,0,1,1,0,1,0,1,1


In [0]:
# 3-grams a nivel palabra
cv = CountVectorizer(ngram_range=(2,2), analyzer="word")
# Extraemos las características del corpus 
cv_matrix = cv.fit_transform(["albert einstein era un cientifico", "la teoría de la relatividad de albert einstein"])

In [0]:
pd.DataFrame(data=cv_matrix.toarray(), columns=cv.get_feature_names())

,albert einstein,de albert,de la,einstein era,era un,la relatividad,la teoría,relatividad de,teoría de,un cientifico
0,1,0,0,1,1,0,0,0,0,1
1,1,1,1,0,0,1,1,1,1,0


### Ejercicio

* Bajar el conjunto de documentos de kaggle https://www.kaggle.com/uciml/sms-spam-collection-dataset  o de
https://drive.google.com/file/d/1HUXG6b7U3_taJw0-Aj6bkl39gdk6h5AK/view?usp=sharing

* Cargarlo el data set

* Construir la matriz tf, tf_idf, y trigramas por palabras

* Seleccionar cinco tweets y clasificarlos con los 5 vecinos más cercanos usando tf_idf con similitud coseno


In [145]:
# Cargamos el dataset como un dataframe de pandas
dataset_df = pd.read_csv("../Files/spam.csv", usecols=[0, 1])
dataset_df.head(10)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"
5,spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, �1.50 to rcv"
6,ham,Even my brother is not like to speak with me. They treat me like aids patent.
7,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,spam,WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,spam,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


In [137]:
dataset_df.count()

v1    5572
v2    5572
dtype: int64

In [139]:
dataset_df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [144]:
print("% clase 'ham' :", round(dataset_df['v1'].value_counts()[0]/dataset_df.count()[0]*100, 1))
print("% clase 'spam':", round(dataset_df['v1'].value_counts()[1]/dataset_df.count()[0]*100, 1))

% clase 'ham' : 86.6
% clase 'spam': 13.4


### 2.4 Word2Vec 

Uno de los problemas de los modelos basados en bolsas de palabras es que no tienen en cuenta características importantes del texto como la estructura, el contexto y el significado. Una alternativa es utilizar métodos de embedding como Word2vec, se trata de un método creado por Google en 2013 que está basado en deep learning y busca transformar las palabras en vectores numéricos dentro de un espacio vectorial que captura información **contextual** y **semántica**. 

● Objetivo: Construir un vector que represente las palabras de un corpus usando “distancias”

● Usa una arquitectura de Red Neuronal para computar eficientemente vectores que representan palabras en grandes conjuntos de datos

● Usa los pesos que aprende la red neuronal o minimización de funciones de
costo, como “Word embedding”

La representación de “Word embedding” puede revelar muchas relaciones
ocultas entre palabras. **Por ejemplo**,
vector ("gato") - vector ("gatito") es similar a vector ("perro") - vector ("cachorro").

<img src="https://mlwhiz.com/images/word2vec.png" alt="word2vec" width="800"/>

Existen dos enfoques para construír un modelo Word2Vec:

* **Skip-gram**: en este modelo, la entrada es la palabra que se quiere representar y las salidas son las palabras que la rodean. Por ejemplo, en la frase "aprendiendo procesamiento de lenguaje natural", si la entrada es la palabra "de", entonces las salidas serán "aprendiendo", "procesamiento", "lenguaje", "natural" si el tamaño del vecindario es 5. La arquitectura de la red neuronal para skip-gram se muestra a continuación:

![skipgram](https://miro.medium.com/max/988/0*xqhh7Gd64VAQ1Sny.png)

* **CBOW**: el modelo *continuous bag-of-words* (CBOW) es muy parecido al modelo skip-gram, la diferencia es que la entrada y la salida se intercambian, es decir, se utiliza el contexto (palabras en el vecindario) para predecir una palabra en específico. La arquitectura se muestra a continuación:

![cbow](https://miro.medium.com/max/413/1*UdLFo8hgsX0a1NKKuf_n9Q.png)

Veamos la implementación de word2vec:

In [0]:
# Para entrenar el modelo word2vec necesitamos un corpus grande
from nltk.corpus import gutenberg
# Importamos módulo para manejar la puntuación
from string import punctuation

In [0]:
# En este caso usaremos la biblia como corpus
bible = gutenberg.sents("bible-kjv.txt")
# Definimos los términos que no vamos a incluír:
remove_terms = punctuation + "0123456789"
print("caracteres inválidos: {}".format(remove_terms))

In [0]:
# Normalizamos el texto
norm_bible = [ [word.lower() for word in sent if word not in remove_terms] for sent in bible]
norm_bible = [' '.join(tok_sent) for tok_sent in norm_bible]
norm_bible = filter(None, normalize_corpus(norm_bible))
norm_bible = [tok_sent for tok_sent in norm_bible if len(tok_sent.split()) > 2]
# imprimimos el número total de filas, un ejemplo de linea raw y procesada
print('Total lines:', len(bible))
print('\nSample line:', bible[10])
print('\nProcessed line:', norm_bible[10])

In [0]:
# tokenizamos oraciones del corpus
wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in norm_bible]
tokenized_corpus[:10]

#### 2.4.1 Gensim

En este caso utilizaremos **[gensim](https://radimrehurek.com/gensim/models/word2vec.html)** para entrenar el modelo **word2vec**:

In [0]:
# Instalamos el módulo word2vec
!pip install gensim

In [0]:
# Importamos word2vec from gemsim
from gensim.models import word2vec

Los siguientes parámetros son utilizados por el modelo Word2vec para construir el modelo:

* feature_size: Determina la dimensión de los vectores de embedding.
* window_context: Es el número de palabras en el vecindario que constituye el contexto.
* min_word_count: Especifica el conteo mínimo de una palabra dentro del corpus para ser incluida dentro del vocabulario.
* sample: este parámetro es usado para el sub-muestreo dentro del algoritmo. Generalmente, los valores entre 0.01 entre 0.0001 funcionan.

In [0]:
feature_size = 100 
window_context = 30 
min_word_count = 1
sample = 1e-3

In [0]:
# definimos el modelo y lo entrenamos
w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size,
                              window=window_context, min_count=min_word_count,
                              sample=sample, iter=50)

In [0]:
# Con gensim podemos encontrar las palabras más similares a determinado término, por ejemplo:
w2v_model.wv.most_similar("egypt", topn=5)

In [0]:
# Podemos encontrar relaciones semánticas entre palabras:
w2v_model.wv.most_similar(positive=['king', 'queen'], negative=['man'])

Para verificar que las palabras similares se representan como vectores similares utilizaremos una técnica de reducción de dimensionalidad para visualizar las representaciones en un espacio bidimensional. En este caso utilizaremos t-SNE, el cual es uno de los algoritmos más utilizados para esta tarea y tiene las siguientes propiedades:

* Se trata de una transformación no-lineal que conserva relaciones líneales entre puntos en un espacio de alta dimensionalidad.
* t-SNE crea una distribución de probabilidad utilizando la distribución normal para definir relaciones entre los puntos en el espacio de alta dimensionalidad.
* Utiliza la distribución t-student en el espacio de baja dimensionalidad con el fin de obtener una representación más dispersa (previene la [maldición de la dimensionalidad o el efecto Hughes](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)).
* Es un método basado en descenso de gradiente que se utiliza para optimizar una función no convexa, por ello, puede converger a un mínimo local y puede requerir varias repeticiones para obtener un buen resultado.

In [0]:
# Para reducir la dimensionalidad del word embedding
from sklearn.manifold import TSNE

In [0]:
# Obtenemos las palabras similares 
similar_words = {search_term: [item[0] for item in w2v_model.wv.most_similar([search_term], topn=5)]
                 for search_term in ['god', 'life', 'death','egypt', 'sin']}

words = sum([[k] + v for k, v in similar_words.items()], [])
wvs = w2v_model.wv[words]

In [0]:
print(words)

In [0]:
# Reducimos la dimensionalidad
tsne = TSNE(n_components=2, random_state=0, n_iter=10000, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(wvs)
labels = words

In [0]:
# Visualizamos 
plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='orange', edgecolors='r')
for label, x, y in zip(labels, T[:, 0], T[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

#### 2.4.2 Spacy

Para utilizar las representaciones vectoriales de las palabras, debemos cargar un modelo de lenguaje más grande que el que hemos venido usando con SpaCy. En el siguiente ejemplo, trabajaremos con el https://spacy.io/models/es#es_core_news_md en **Español**. 

In [0]:
!python -m spacy download es_core_news_md

In [0]:
# Importamos spaCy y cargamos los modelos
import spacy
import es_core_news_md
nlp = es_core_news_md.load()  # Para español - la versión md (74MB) incluye word embeddings

In [0]:
nlp(u'perro').vector

Los objetos Doc y Span tienen vectores, derivados de los promedios de los vectores de tokens individuales. Esto permite comparar similitudes entre documentos completos.

In [0]:
doc = nlp(u'El perro blanco estaba ladrando en el parque.')

doc.vector

**Simulitud de vectores:**

Más info: https://spacy.io/usage/vectors-similarity

In [0]:
tokens = nlp(u'tigre gato perro mascota felino')

# Combinaciones
for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

Los **antónimos** no son necesariamente vectores muy diferentes. A veces aparecen en los mismos contextos por lo que podrían tener vectores similares.

In [0]:
tokens = nlp(u'amar odiar')

# Combinaciones
for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

In [0]:
tokens = nlp(u'frío calor')

# Combinaciones
for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

**Aritmética de vectores**


In [0]:
from scipy import spatial

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

profe = nlp.vocab['profesor'].vector
hombre = nlp.vocab['hombre'].vector
mujer = nlp.vocab['mujer'].vector

# Podemos encontrar el vector más cercano en el vocabulario para el resultado "profe" - "hombre" + "mujer"
nuevo_vector = profe - hombre + mujer
computed_similarities = []

for word in nlp.vocab:
    # Ignore words without vectors and mixed-case words:
    if word.has_vector:
        if word.is_lower:
            if word.is_alpha:
                similarity = cosine_similarity(nuevo_vector, word.vector)
                computed_similarities.append((word, similarity))

computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])

print([w[0].text for w in computed_similarities[:10]])

### 2.5 FastText

Se trata de una extensión de Word2Vec que fue propuesta por facebook en el 2016. En este caso, en lugar de utilizar palabras individuales, los textos se dividen en n-gramas de caracteres. Por ejemplo, los 3-gramas para la palabra "universidad" son: "uni", "niv", "ive", "ver", "ers", "rsi", "sid", "ida", "dad". La representación de una palabra es la suma de estos n-gramas y es utilizada para entrenar una red neuronal. Este enfoque permite resaltar palabras raras, fundamentalmente, porque es muy probable que sus n-gramas también aparezcan en otras palabras.


In [0]:
# importamos el modelo FastText
from gensim.models import FastText

In [0]:
ft_model = FastText(tokenized_corpus, size=feature_size, # Volvemos a usar el corpus de la Biblia
                  window=window_context, min_count=min_word_count,
                  iter=50, workers=4,sg=1)

In [0]:
similar_words = {search_term: [item[0] for item in w2v_model.wv.most_similar([search_term], topn=5)]
                 for search_term in ['god', 'jesus', 'noah','egypt', 'john', 'gospel', 'moses','famine']}
similar_words

## 3. Ejercicio

* Siga los pasos utilizados en word2vec (con GenSim) para visualizar y encontrar la representación vectorial de los textos con FastText

## Referencias

* [1] Rahul Agarwal. NLP Learning Series: Part 1 - Text Preprocessing Methods for Deep
Learning. 2019. url: https://mlwhiz.com/blog/2019/01/17/deeplearning%7B%
5C_%7Dnlp%7B%5C_%7Dpreprocess/.
* [2] M.J. Ashwin. Next Word Prediction Using Markov Model. 2018. url: https://
medium.com/ymedialabs-innovation/next-word-prediction-using-markovmodel-570fc0475f96.
* [3] FacebookOpenSource. FastText: Library for efficient text classification and representation learning. 2019. url: https://fasttext.cc/.
* [4] Shreya Ghelani. From Word Embeddings to Pretrained Language Models A New
Age In NLP - Part-1. 2016. url: https://towardsdatascience.com/from-wordembeddings- to- pretrained- language- models- a- new- age- in- nlp- part- 1-
7ed0c7f3dfc5.
* [5] Steeve Huang. Word2Vec and FastText Word Embedding with Gensim. 2018. url:
https : / / towardsdatascience . com / word - embedding - with - word2vec - and -
fasttext-a209c1d3e12c.
* [6] Christopher D. Manning, Prabhakar Raghavan, and Hinrich Schütze. Introduction
to Information Retrieval. New York, NY, USA: Cambridge University Press, 2008.
isbn: 0521865719, 9780521865715.
* [7] Gonzalo RuizdeVilla. Introducción a Word2vec (skip gram model). 2018. url: https:
//medium.com/@gruizdevilla/introducci%7B%5C’%7Bo%7D%7Dn-a-word2vecskip-gram-model-4800f72c871f.